# Introduction

In this notebook we introduce the methodology of using an open source language model (classifier) to classify strings into a set of arbitrary categories. 

The language model we will use is pretrained on a large dataset (just like the models underpinning OpenAI's recently famous ChatGPT interface). This model in particular has been trained and released by Facebook and has been optimized on the task of classification instead of text generation, but the underlying techniques are similar. It therefore is able to perform the tasks we want directly out-of-the-box without any further tweaking!

Classification is the act of assigning a probability between one and zero to a (set of) label(s) being applicable to a string of text. 

To give an example, the sentence "The sun is out today." seems pretty happy. If a person were asked to give it a score on "happiness", they might assign it a score of 90%. But what about the sentence "It is raining outside.."? That perhaps should score only 10%, if not lower. The language model we use will be able to draw similar conclusions automatically based on it's understanding of the English language.

Let's get started!

# Setup

In [29]:
# Import the pipeline helper from the transformers package, which we will use to load our model
from transformers import pipeline
import pandas as pd

In [18]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")


GPU is available


In [2]:
# Download and instantiate the facebook/bart-large-mlni model and pipeline. The first time this cell runs, it downloads a large file containing the model weights (1.5GB of parameters all in all!). 
# Let it finish and from then on it will be cached on disk.

# Stay aware that this model instantiation uses a lot of memory/RAM, as it has to load the full 1.5GB of model parameters. If you load the model as below in several notebooks at the same time, you might overload your box. 
# To avoid this, when you're done with a notebook, close it's "kernel" on the left side of the Jupyterlabs interface (The second icon. circle with a square inside, selects the active kernels). This unloads the model from memory.

c = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# Testing whether a label is applicable

Let's see if we can replicate the ideas above. Is the sentence "The sun is out today!" indeed classifiably happy?

To do so, we use the defined interface of our classifier, which takes (at least) the following arguments: a sentence, a list of labels.

Perhaps it's easier to just look at an example!

In [3]:
c('The sun is out today.', ['Happy']).get('scores')

[0.9586013555526733]

Great result! It looks like our sentence is indeed very happy. With a score of almost 96%, it doesn't get much happier than that.

Challenge: Can you find a way to modify the sentence such that it is even happier?

In [4]:
# Modify the sentence to something even happier
c('The sun is out today and I love sunny days.', ['Happy'])

{'sequence': 'The sun is out today and I love sunny days.',
 'labels': ['Happy'],
 'scores': [0.9684773683547974]}

And how about our "unhappy" sentence, will it indeed agree?

In [5]:
c('It is raining outside..', ['Happy'])

{'sequence': 'It is raining outside..',
 'labels': ['Happy'],
 'scores': [0.00019142501696478575]}

Nobody likes rain..

# Multiple labels

If that's not a happy sentence, then perhaps we can conclude it's a sad one? Does our model agree?

We can ask it to make a choice between several available labels.

In [6]:
c('It is raining outside..', ['Happy', 'Sad'])

{'sequence': 'It is raining outside..',
 'labels': ['Sad', 'Happy'],
 'scores': [0.9559351801872253, 0.044064849615097046]}

The mathematics of the model work out slightly differently to before for multi-labeling, but the conclusion remains the same, it certainly considers the sentence to be much more sad than happy.

What about other labels, like wet, dry, high, low and colorful or gray?

c('It is raining outside..', ['Happy', 'Sad', 'Wet', 'Dry', 'High', 'Low', 'Colorful', 'Gray'])

Hard question, but a clear and agreeable answer, it's more Wet than any other of those labels. But isn't rain both wet and sad?

To answer that question we can ask the model to assign a probability per label, rather than forcing it to make a choice. Note that in the above result, all label probabilities add up to 100%.

In [7]:
c('It is raining outside..', ['Happy', 'Sad', 'Wet', 'Dry', 'High', 'Low', 'Colorful', 'Gray'], multi_label=True)

{'sequence': 'It is raining outside..',
 'labels': ['Wet', 'Sad', 'Low', 'Gray', 'High', 'Colorful', 'Dry', 'Happy'],
 'scores': [0.9990817308425903,
  0.8136842250823975,
  0.7355229258537292,
  0.4133484959602356,
  0.038622740656137466,
  0.0017680926248431206,
  0.0003273731854278594,
  0.00019142501696478575]}

This makes sense!

# Edge cases

Negatiation. Surely, if rain is bad, then the opposite should be good?

In [8]:
c('It is not raining outside..', ['Happy', 'Sad'])

{'sequence': 'It is not raining outside..',
 'labels': ['Happy', 'Sad'],
 'scores': [0.853407084941864, 0.1465929001569748]}

It depends on the context though, and the model can handle that pretty well too.

In [9]:
c('After the drought it is finally raining outside..', ['Happy', 'Sad'])

{'sequence': 'After the drought it is finally raining outside..',
 'labels': ['Happy', 'Sad'],
 'scores': [0.7387925982475281, 0.26120737195014954]}

# Multiple sentences/strings

We can also ask the model to classify multiple senteces in one go. That might be useful when you use this model for your trading strategy later.

In [10]:
c(['The sun is out today.', 'It is raining outside..'], ['Happy', 'Sad'])

[{'sequence': 'The sun is out today.',
  'labels': ['Happy', 'Sad'],
  'scores': [0.9836675524711609, 0.016332462430000305]},
 {'sequence': 'It is raining outside..',
  'labels': ['Sad', 'Happy'],
  'scores': [0.9559351801872253, 0.044064849615097046]}]

# Free-form exploration

Below, try a few different examples, different labels, sentence structures, and so on, to get a feel for that the model can and can't handle very well.

In [11]:
c('Chemistry is the best subject in the world!', ['Love for Chemistry', 'Hate for Chemistry'])

{'sequence': 'Chemistry is the best subject in the world!',
 'labels': ['Love for Chemistry', 'Hate for Chemistry'],
 'scores': [0.9888017177581787, 0.011198299005627632]}

In [25]:
NVIDIA = 'NVIDIA\'s contributions to a major industry collaboration have given the stock a boost. #IndustryCollaboration #GraphicsChip'

In [26]:
Pfizer = 'Pfizer faces backlash over possible closure of regional office. #PharmaNews #RegionalOffice'

In [27]:
ambiguous ='A recent study found that most CEOs only read business books. That explains a lot.'

In [33]:
sequences = []
profitable_scores = []
not_profitable_scores = []

# Loop through each item in the social media feed
for item in [Pfizer, NVIDIA, ambiguous]:
    # Get the sentiment analysis result
    result = c(item,['Profitable','Not Profitable'])
    
    # Extract the needed information
    sequences.append(result['sequence'])
    temp_scores = {'Profitable':None,'Not Profitable':None}
    for label, score in zip(result['labels'],result['scores']):
        temp_scores[label] = score
    # Append the scores to their respective lists
    profitable_scores.append(temp_scores['Profitable'])
    not_profitable_scores.append(temp_scores['Not Profitable'])


# Create a DataFrame with the extracted information
df = pd.DataFrame({
    'sequence': sequences,
    'Profitable': profitable_scores,
    'Not Profitable': not_profitable_scores
})

,sequence,Profitable,Not Profitable
0,Pfizer faces backlash over possible closure of...,0.195984,0.804016
1,NVIDIA's contributions to a major industry col...,0.972254,0.027746
2,A recent study found that most CEOs only read ...,0.775821,0.224179
